# 2018
La Encuesta Nacional de la Dinámica Demográfica (ENADID) 2018 se llevó a cabo con la finalidad de proporcionar información estadística relacionada con el nivel y comportamiento de los componentes de la dinámica demográfica: fecundidad, mortalidad y migración (interna e internacional); así como otros temas referidos a la población, los hogares y las viviendas.

## Descarga de Datos
Los datos fueron descargados de forma manual, se prevee automatizarlo en un futuro.

## Manejo de Datos
El manejo de datos se divide en 3 campos. 
* Análisis de Datos: Observar los datos a fin de buscar propiedades útiles y pensar como separarlas del resto del documento. 
* Manipulación de Datos: La manipulación de la base da datos original para obtener la parte que nos interesa. 
* Almacenaje de Datos. Pensar el formato que le daremos para el guardado. 

El Análisis y la Manipulación de Datos se realizaron fuera de esta libreta, de igual forma se anexa el resultado final.

In [12]:
import pandas as pd  # Importación de Librerias.
import numpy as np


def comparelists(listaA: list, listaB: list):  # Función auxiliar para comparar dos listas. 
    d_index = []
    if len(listaA) == len(listaB):
        for i in range(len(listaA)):
            if listaA[i] != listaB[i]:
                d_index.append(i)
    return d_index


def t_header(data: pd.DataFrame):  ## * Derivadas de la estructura del DataFrame inicial 
    indicate_remove = data[
    'INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación '].isna()
    indicate_remove = list(indicate_remove)
    remove = 0
    t = 0
    while remove < 3:
        if indicate_remove[t] == True:
            remove += 1
        t += 1
    return t

def remove_header(t, data: pd.DataFrame):  ## * 
    remove_index = np.arange(0, t)
    data = data.drop(index = remove_index)

def couple_cols(actual: pd.DataFrame, new: pd.DataFrame):  # * 
    now_cols = list(actual.columns)
    new_cols = list(new.columns)
    if len(now_cols) != len(new_cols):
        return -1
    for i in range(len(now_cols)):
        now_cols[i] = str(now_cols[i]) 
        new_cols[i] = str(new_cols[i])
    mod_cols = comparelists(now_cols, new_cols)
    new_cols = {new_cols[i] : now_cols[i] for i in mod_cols}
    new = new.rename(columns=new_cols)
    return new

def df_blocks(ldf):  # * 
    superdf = pd.DataFrame()
    for q, df_test in enumerate(ldf):
        df_cols = list(df_test.iloc[1,:])
        if df_cols[2] == 'Nemónico_PE':
            df_cols[2] = 'Mnemónico_PE'
        df_test.columns = df_cols
        df_name = df_test.iloc[0, 1]
        df_test.loc[:, ['SubNombre']] = df_name
        if q == 0:
            df_test = df_test.drop(columns = df_test.columns[0]).iloc[4: ,:]
        else:
            df_test = df_test.drop(columns = df_test.columns[0]).iloc[3: ,:]
            df_test = couple_cols(superdf, df_test)
        df_clean = df_test.dropna(axis= 0, how='all').ffill()
        superdf = pd.concat([superdf, df_clean])
    return superdf
def enadid2018_all(save_to: str):
    all_df = pd.DataFrame()
    enadid = pd.read_excel('data\\fd_enadid_2018_2014_2018.xlsx', sheet_name=None)
    dictsheets = enadid.keys()
    for sheet in dictsheets:  
        print(sheet)
        tab_df = enadid[sheet]
        tab_df['Unnamed: 0'] = tab_df.isna().sum(axis = 1)
        n_cols = tab_df['Unnamed: 0'].max()
        sub_tables = tab_df[tab_df['Unnamed: 0'] == (n_cols - 1)]
        sub_tables = sub_tables[~sub_tables['INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación '].isna()]
        names = sub_tables['INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación '].to_list()
        sub_index = np.array(sub_tables.index)
        sub_index = sub_index[sub_index > t_header(tab_df)]
        subdfs = []
        for i in range(len(sub_index) - 1):
            subdfs.append(tab_df.iloc[sub_index[i]:sub_index[i + 1],:])
        if sub_index[i + 1] == sub_index[-1]:
            subdfs.append(tab_df.iloc[sub_index[i + 1]:,:])
        sub_all_df = df_blocks(subdfs)
        sub_all_df.set_index('ID',inplace=True)
        sub_all_df['Hoja'] = sheet
        all_df = pd.concat([all_df, sub_all_df])
    all_df.to_excel(save_to, sheet_name='toPython')

def enadid2018_tabs(save_to : str):
    writer = pd.ExcelWriter(save_to, engine='xlsxwriter')
    enadid = pd.read_excel('data\\fd_enadid_2018_2014_2018.xlsx', sheet_name=None)
    dictsheets = enadid.keys()
    for sheet in dictsheets:  
        print(sheet)
        tab_df = enadid[sheet]
        tab_df['Unnamed: 0'] = tab_df.isna().sum(axis = 1)
        n_cols = tab_df['Unnamed: 0'].max()
        sub_tables = tab_df[tab_df['Unnamed: 0'] == (n_cols - 1)]
        sub_tables = sub_tables[~sub_tables['INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación '].isna()]
        names = sub_tables['INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación '].to_list()
        sub_index = np.array(sub_tables.index)
        sub_index = sub_index[sub_index > t_header(tab_df)]
        subdfs = []
        for i in range(len(sub_index) - 1):
            subdfs.append(tab_df.iloc[sub_index[i]:sub_index[i + 1],:])
        if sub_index[i + 1] == sub_index[-1]:
            subdfs.append(tab_df.iloc[sub_index[i + 1]:,:])
        sub_all_df = df_blocks(subdfs)
        sub_all_df.set_index('ID',inplace=True)
        sub_all_df['Hoja'] = sheet
        sub_all_df.to_excel(writer, sheet_name= sheet)
    writer.close()


Finalmente, veremos el resultado en la carpeta result.
### Almacenaje de Datos. 

#### Almacenaje NO programatico. 

Con no programatico hacemos referencia a que el resultado será revisado por una persona, por lo tanto conviene más que la hojas se mantengan separadas. 

In [13]:
dir_save = 'results\\'
filename = 'enadid2018_tabs.xlsx'
file_path = dir_save + filename
enadid2018_tabs(file_path)

TVIVIENDA
THOGAR
TSDEM
TMIGRANTE
TFECHISEMB
TMUJER1
TMUJER2


#### Almacenaje programatico. 

Aquí, haremos la consulta mediante `Python`  por lo tanto conviene que todos los descriptores se encuentren un solo archivo correctamente etiquetado. (Para eso la creación de las nuevas columnas.)

In [14]:
dir_save = 'results\\'
filename = 'enadid2018_all.xlsx'
file_path = dir_save + filename
enadid2018_all(file_path)

TVIVIENDA
THOGAR
TSDEM
TMIGRANTE
TFECHISEMB
TMUJER1
TMUJER2


Ahora, viajaremos en el tiempo. Regresando a 2014 y recuperando sus descriptores.